## Exploring the scRNA-seq data from Parkinson's dataset
Want to get a sense of the cell types available and how I can
go about converting the format given by the dataset into
a format that is compatible with Geneformer

[Link to dataset](https://singlecell.broadinstitute.org/single_cell/study/SCP1402/a-molecular-census-of-midbrain-dopaminergic-neurons-in-parkinsons-disease-preprint-data#study-download)

#### Files from dataset

[Link to barcodes table](https://singlecell.broadinstitute.org/single_cell/data/public/SCP1402/a-molecular-census-of-midbrain-dopaminergic-neurons-in-parkinsons-disease-preprint-data?filename=Homo_bcd.tsv.gz) (2.61 MB) - Homo_bcd.tsv.gz

[Link to genes table](https://singlecell.broadinstitute.org/single_cell/data/public/SCP1402/a-molecular-census-of-midbrain-dopaminergic-neurons-in-parkinsons-disease-preprint-data?filename=Homo_features.tsv.gz) (288 KB) - Homo_features.tsv.gz

[Link to cell metadata](https://singlecell.broadinstitute.org/single_cell/data/public/SCP1402/a-molecular-census-of-midbrain-dopaminergic-neurons-in-parkinsons-disease-preprint-data?filename=METADATA_PD.tsv.gz) (3.8 MB) - METADATA_PD.tsv.gz

[Link to barcodes annotated as oligodendrocytes](https://singlecell.broadinstitute.org/single_cell/data/public/SCP1402/a-molecular-census-of-midbrain-dopaminergic-neurons-in-parkinsons-disease-preprint-data?filename=olig_UMAP.tsv) (17 MB) - olig_UMAP.tsv

[Link to count matrix](https://singlecell.broadinstitute.org/single_cell/data/public/SCP1402/a-molecular-census-of-midbrain-dopaminergic-neurons-in-parkinsons-disease-preprint-data?filename=Homo_matrix.mtx.gz) (4.74GB) - Homo_matrix.mtx.gz

** For some reason, the above files are downloaded with the ".tsv" extension but are actually gzipped files

#### Ensembl ID mappings

[Link to Ensembl ID mappings](http://useast.ensembl.org/biomart/martview/26693be23a3ce39a57e123396eda8f7a?VIRTUALSCHEMANAME=default&ATTRIBUTES=hsapiens_gene_ensembl.default.feature_page.ensembl_gene_id|hsapiens_gene_ensembl.default.feature_page.ensembl_transcript_id|hsapiens_gene_ensembl.default.feature_page.external_gene_name&FILTERS=&VISIBLEPANEL=attributepanel) (10.8 MB) - mart_export.txt

In [21]:
import scipy.io
import pandas as pd
import numpy as np
import anndata as ad
import gzip

### Import all of the column and row attributes of the count matrix

In [22]:
# Gene features
with gzip.open('Homo_features.tsv', 'rt') as f:
    genes = pd.read_csv(f, delimiter='\t', header = None)

ensembl_mapping = pd.read_table("mart_export.txt").\
                  set_index("Gene name").to_dict()['Gene stable ID']

# Barcodes
with gzip.open('Homo_bcd.tsv', 'rt') as f:
    barcodes = pd.read_csv(f, delimiter='\t', header = None)

### Import metadata for filtering to barcodes of interest

In [23]:
# Metadata for each cell
with gzip.open('METADATA_PD.tsv.gz', 'rt') as f:
    metadata_df = pd.read_csv(f, delimiter='\t')
metadata_df.head()

/var/folders/mz/2gg71my1441_g_c9np4_j2kh0000gn/T/ipykernel_79509/1294798127.py:3: DtypeWarning: Columns (3,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(f, delimiter='\t')


,NAME,libname,biosample_id,donor_id,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label,sex,date,Donor_Age,Donor_PMI,Status,Cause_of_Death,FACS_Classification
0,TYPE,group,group,group,group,group,group,group,group,group,group,group,group,group,numeric,numeric,group,group,group
1,pPDCN4340DAPIA030419_AATGCCACACAAGCAG-1,pPDCN4340DAPIA030419,pPDCN4340DAPIA030419,4340,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0001873,caudate nucleus,EFO_0009901,10x 3' v1,male,190304,47,12.5,Ctrl,Esophageal cancer with liver mets,Negative
2,pPDCN4340DAPIA030419_AGACTCAGTCACAATC-1,pPDCN4340DAPIA030419,pPDCN4340DAPIA030419,4340,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0001873,caudate nucleus,EFO_0009901,10x 3' v1,male,190304,47,12.5,Ctrl,Esophageal cancer with liver mets,Negative
3,pPDCN4340DAPIA030419_ATACTTCCAGCGTTGC-1,pPDCN4340DAPIA030419,pPDCN4340DAPIA030419,4340,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0001873,caudate nucleus,EFO_0009901,10x 3' v1,male,190304,47,12.5,Ctrl,Esophageal cancer with liver mets,Negative
4,pPDCN4340DAPIA030419_ATATCCTGTGTGTTTG-1,pPDCN4340DAPIA030419,pPDCN4340DAPIA030419,4340,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0001873,caudate nucleus,EFO_0009901,10x 3' v1,male,190304,47,12.5,Ctrl,Esophageal cancer with liver mets,Negative


### Oligodendrocytes are the most abundant cell type
For the purpose of training an MVP model we may want to focus on cell type with most training examples

In [24]:
# Barcodes annotated as oligodendrocytes
oligos = pd.read_csv("olig_UMAP.tsv", sep = "\t")
oligo_barcodes = oligos.iloc[1:]['NAME'].tolist()

/var/folders/mz/2gg71my1441_g_c9np4_j2kh0000gn/T/ipykernel_79509/856640324.py:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  oligos = pd.read_csv("olig_UMAP.tsv", sep = "\t")


In [25]:
# Filter metadata_df to metadata of interest
metadata_df_filtered = metadata_df.query("species__ontology_label == 'Homo sapiens'\
         & disease__ontology_label.isin(['normal','Parkinson disease'])\
         & organ__ontology_label == 'substantia nigra pars compacta'\
         & NAME.isin(@oligo_barcodes)")

In [26]:
metadata_df_filtered[['donor_id','disease__ontology_label']].drop_duplicates()

,donor_id,disease__ontology_label
22497,3482,normal
27620,3346,normal
33309,3482,normal
37739,3346,normal
42681,3873,Parkinson disease
44675,5610,normal
49799,3322,normal
78539,3345,normal
93844,4956,normal
117212,6173,normal


In [27]:
metadata_df_filtered['donor_id'].unique()

array(['3482', '3346', 3482, 3346, 3873, 5610, 3322, 3345, 4956, 6173,
       3887, 4560, 4568, 2142, 1963, 3298, '1963', '3298'], dtype=object)

In [28]:
# How many unique donors?
metadata_df_filtered['donor_id'].nunique()

18

In [30]:
len(metadata_df_filtered)

143582

In [29]:
# Get barcodes filtered to metadata of interest
barcodes_of_interest = pd.Index(metadata_df_filtered['NAME'])

# Create dict for matching barcodes to disease
disease_dict = metadata_df_filtered.set_index('NAME')['disease__ontology_label']

# Create dict for matching barcodes to donor_id
metadata_df_filtered['donor_id'] = metadata_df_filtered['donor_id'].astype(str)
donor_dict = metadata_df_filtered.set_index('NAME')['donor_id']

/var/folders/mz/2gg71my1441_g_c9np4_j2kh0000gn/T/ipykernel_79509/1160797698.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata_df_filtered['donor_id'] = metadata_df_filtered['donor_id'].astype(str)


### The format required by Geneformer is described [here](https://huggingface.co/ctheodoris/Geneformer/blob/main/examples/tokenizing_scRNAseq_data.ipynb)
I will convert the count matrix into the appropriate format.

In [ ]:
# Load the MTX file
with gzip.open("Homo_matrix.mtx.gz", "rt") as f:
    mtx = scipy.io.mmread(f).tocsc()

# Create an AnnData object
adata = ad.AnnData(X=mtx.T)

# Map gene names to Ensembl IDs
adata.var['gene_name'] = genes[0].values
adata.var['ensembl_id'] = adata.var['gene_name'].map(ensembl_mapping)

# Filter AnnData object to barcodes of interest
adata.obs['barcode'] = barcodes[0].values
adata = adata[adata.obs['barcode'].isin(barcodes_of_interest)]

# Add columns necessary for Geneformer fine-tuning
adata.obs['n_counts'] = np.sum(adata.X, axis=1).A1
adata.obs['donor_id'] = adata.obs['barcode'].map(donor_dict)
adata.obs['disease_status'] = adata.obs['barcode'].map(disease_dict)

In [ ]:
!mkdir PD_data_directory/

In [ ]:
# Save the AnnData object as an H5 file
adata.write_h5ad('PD_data_directory/PD.h5ad')

### Tokenization for Geneformer

In [ ]:
from geneformer import TranscriptomeTokenizer
tk = TranscriptomeTokenizer({"disease_status":"disease_status",
                             "donor_id":"donor_id"}, nproc=16)
tk.tokenize_data(data_directory="PD_data_directory", 
                 output_directory="PD_data", 
                 output_prefix="PD", 
                 file_format="h5ad")

### Verify

In [2]:
from datasets import load_from_disk
data = load_from_disk("PD_data/PD.dataset")
data.data

/Users/erykkropiwnicki/.pyenv/versions/retrobio/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ConcatenationTable
input_ids: list<item: int16>
  child 0, item: int16
disease_status: string
donor_id: string
length: int64
----
input_ids: [[[4113,15947,16985,15366,4067,...,14787,18513,80,1680,14231],[4113,15947,16985,8937,15366,...,8836,6952,5575,5830,6909],...,[9371,16985,13462,15947,1932,...,11736,5613,18001,7129,3758],[9371,9291,16985,13462,11952,...,234,8439,8967,407,6272]],[[9371,9291,15947,16985,13923,...,13047,9413,318,17182,12696],[15947,16985,11656,1106,13462,...,3758,17269,4214,16923,16473],...,[15947,11656,948,16985,9855,...,739,8330,4521,9301,9539],[9371,16985,15947,9183,11952,...,7936,10352,16018,11909,2120]],...,[[16985,13462,9291,15947,1106,...,8041,171,3356,1169,2838],[15947,4067,15287,16985,14609,...,20421,10862,11556,10859,4189],...,[1106,9291,13462,16985,15947,...,17247,20499,12254,5381,12030],[1106,11656,15287,4067,9291,...,17247,17326,15458,6367,8659]],[[4067,9291,16985,9939,15947,...,10152,12820,5381,6367,14485],[15947,16985,11656,24325,14258,...,3160,16913,17